In [3]:
import Annotator
import pybedtools
import os
import pandas as pd

pd.set_option('display.max_colwidth', 5000)
from tqdm import tnrange, tqdm_notebook

In [8]:
d = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
brian_annotated = os.path.join(
    d,
    'WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed.brianannotated3'
)
eric_annotated = os.path.join(
    d, 'WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed.annotated'
)

def parse_feature(row, col):
    """
    This is identical to the 'parse_feature' function before, 
    but now we look at the 'name' instead of 'priority'
    
    Parses each annotated row and returns the 'col' value 
    Or returns INTERGENIC if the row has no other annotation.
    
    col : int
    
    """
    if 'INTERGENIC' not in row['priority'].upper():
        return row['priority'].split(':')[col]
    return 'INTERGENIC'
    

def return_differences(brian_annotated, eric_annotated):
    brian_bedhead = ['chrom','start','end','p','f','strand','priority','annotation']
    eric_bedhead = ['chrom','start','end','p','f','strand','priority','gene']

    brian = pd.read_table(brian_annotated, names=brian_bedhead)
    eric = pd.read_table(eric_annotated, names=eric_bedhead)

    brian['region'] = brian['annotation'].apply(lambda x: x.split(':')[4].upper())
    eric['region'] = eric['annotation'].apply(lambda x: x.split('|')[0].upper())
    merged = pd.merge(brian, eric, how='left', on=['chrom','start','end','p','f','strand'])
    diffed = merged[merged['region_x']!=merged['region_y']]
    return diffed

In [9]:
return_differences(brian_annotated, eric_annotated)

KeyError: 'annotation'

In [17]:
d = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
bed = os.path.join(d,'ms1_ctrl_r1_01.basedon_ms1_ctrl_r1_01.peaks.l2inputnormnew.bed.compressed.bed.brian')

brian_bedhead = ['chrom','start','end','p','f','strand','priority','annotation']
brian = pd.read_table(bed, names=brian_bedhead)
def get_region(row):
    if ':' in row['priority']:
        return row['priority'].split(':')[4].upper()
    else:
        return 'intergenic'
brian['region'] = brian.apply(get_region, axis=1)
brian.head()

,chrom,start,end,p,f,strand,priority,annotation,region
0,chr7,100550762,100550861,79.930184,6.231398,+,ENST00000483133.1:100550330:100552774:+:noncoding_exon:ENSG00000169894.13:MUC3A:retained_noncoding_intron,ENSG00000169894.13:100547187:100611118:+:gene:ENSG00000169894.13:MUC3A:protein_coding|ENST00000483133.1:100550330:100611118:+:transcript:ENSG00000169894.13:MUC3A:retained_intron|ENST00000483133.1:100550330:100552774:+:exon:ENSG00000169894.13:MUC3A:retained_intron,NONCODING_EXON
1,chr7,141036595,141036717,66.282749,7.621372,+,ENST00000565468.1:140912505:141137407:+:intron:ENSG00000261115.1:TMEM178B:protein_coding,ENST00000565468.1:140774032:141180180:+:transcript:ENSG00000261115.1:TMEM178B:protein_coding|ENST00000565468.1:140912505:141137407:+:intron:ENSG00000261115.1:TMEM178B:protein_coding|ENSG00000261115.1:140774032:141180180:+:gene:ENSG00000261115.1:TMEM178B:protein_coding,INTRON
2,chr7,100458401,100458539,64.599237,6.625276,+,ENST00000418037.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding,ENST00000418037.1:100450355:100458802:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000418037.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000487651.1:100456358:100464631:+:transcript:ENSG00000146828.13:SLC12A9:retained_intron|ENST00000487651.1:100456358:100459206:+:exon:ENSG00000146828.13:SLC12A9:retained_intron|ENST00000475623.1:100456659:100458566:+:transcript:ENSG00000146828.13:SLC12A9:processed_transcript|ENST00000475623.1:100458139:100458566:+:exon:ENSG00000146828.13:SLC12A9:processed_transcript|ENST00000416675.1:100450399:100458852:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000416675.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000275729.3:100450358:100463007:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000275729.3:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000467972.1:100454416:100464631:+:transcript:ENSG00000146828.13:SLC12A9:retained_intron|ENST00000467972.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:retained_intron|ENST00000540482.1:100450341:100462980:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000540482.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENSG00000146828.13:100424442:100464631:+:gene:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000415287.1:100450359:100463157:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000415287.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000354161.3:100450375:100464630:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000354161.3:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000475687.1:100453063:100464631:+:transcript:ENSG00000146828.13:SLC12A9:retained_intron|ENST00000475687.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:retained_intron|ENST00000428758.1:100450358:100463007:+:transcript:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000428758.1:100457855:100458759:+:intron:ENSG00000146828.13:SLC12A9:protein_coding|ENST00000448342.1:100450342:100458884:+:transcript:ENSG00000146828.13:SLC12A9:nonsense_mediated_decay|ENST00000448342.1:100457855:100458633:+:intron:ENSG00000146828.13:SLC12A9:nonsense_mediated_decay,INTRON
3,chr7,141037023,141037096,61.406161,7.515959,+,ENST00000565468.1:140912505:141137407:+:intron:ENSG00000261115.1:TMEM178B:protein_coding,ENST00000565468.1:140774032:141180180:+:transcript:ENSG00000261115.1:TMEM178B:protein_coding|ENST00000565468.1:140912505:141137407:+:intron:ENSG00000261115.1:TMEM178B:protein_coding|ENSG00000261115.1:140774032:141180180:+:gene:ENSG00000261115.1:TMEM178B:protein_coding,INTRON
4,chr7,141037096,141037172,59.550927,7.473273,+,ENST00000565468.1:140912505:141137407:+:intron:ENSG00000261115.1:TMEM178B:protein_coding,ENST00000565468.1:140774032:141180180:+:transcript:ENSG00000261115.1:TMEM178B:protein_coding|ENST0000

In [18]:
wd = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
annotated_bedhead = ['chrom','start','end','p','f','strand','annotation','gene']
eric = pd.read_table(
    os.path.join(
        wd, 'ms1_ctrl_r1_01.basedon_ms1_ctrl_r1_01.peaks.l2inputnormnew.bed.compressed.bed.annotated'
    ),
    names=annotated_bedhead
)
eric['region'] = eric['annotation'].apply(lambda x: x.split('|')[0].upper())
eric.head()

,chrom,start,end,p,f,strand,annotation,gene,region
0,chr7,100550762,100550861,79.930184,6.231398,+,noncoding_exon|contained|ENSG00000169894.13,ENSG00000169894.13,NONCODING_EXON
1,chr7,141036595,141036717,66.282749,7.621372,+,intron|contained|ENSG00000261115.1,ENSG00000261115.1,INTRON
2,chr7,100458401,100458539,64.599237,6.625276,+,intron|contained|ENSG00000146828.13,ENSG00000146828.13,INTRON
3,chr7,141037023,141037096,61.406161,7.515959,+,intron|contained|ENSG00000261115.1,ENSG00000261115.1,INTRON
4,chr7,141037096,141037172,59.550927,7.473273,+,intron|contained|ENSG00000261115.1,ENSG00000261115.1,INTRON


In [19]:
merged = pd.merge(brian, eric, how='left', on=['chrom','start','end','strand'])
diffed = merged[merged['region_x']!=merged['region_y']]
diffed

,chrom,start,end,p_x,f_x,strand,priority,annotation_x,region_x,p_y,f_y,annotation_y,gene,region_y
116,chr7,142424278,142424367,0.755104,2.093895,+,INTERGENIC,INTERGENIC,intergenic,0.755104,2.093895,intergenic,NaN,INTERGENIC
723,chr7,121684475,121684476,0.000000,0.678857,+,ENST00000393386.2:121684476:121684622:+:CDS:ENSG00000106278.7:PTPRZ1:protein_coding,ENST00000393386.2:121513143:121702090:+:transcript:ENSG00000106278.7:PTPRZ1:protein_coding|ENST00000393386.2:121684476:121684622:+:exon:ENSG00000106278.7:PTPRZ1:protein_coding|ENST00000393386.2:121684476:121684622:+:CDS:ENSG00000106278.7:PTPRZ1:protein_coding|ENSG00000106278.7:121513143:121702090:+:gene:ENSG00000106278.7:PTPRZ1:protein_coding|ENST00000449182.1:121684476:121684622:+:exon:ENSG00000106278.7:PTPRZ1:protein_coding|ENST00000449182.1:121684476:121684622:+:CDS:ENSG00000106278.7:PTPRZ1:protein_coding|ENST00000449182.1:121513415:121701415:+:transcript:ENSG00000106278.7:PTPRZ1:protein_coding,CDS,0.000000,0.678857,intergenic,NaN,INTERGENIC
921,chr7,99965145,99965212,0.000000,-0.321143,+,ENST00000432297.2:99965153:99965240:+:5UTR:ENSG00000085514.11:PILRA:protein_coding,ENST00000432297.2:99965153:99996925:+:transcript:ENSG00000085514.11:PILRA:protein_coding|ENST00000432297.2:99965153:99965240:+:exon:ENSG00000085514.11:PILRA:protein_coding|ENST00000432297.2:99965153:99965240:+:5UTR:ENSG00000085514.11:PILRA:protein_coding|ENST00000448382.1:99964972:99965356:+:exon:ENSG00000121716.14:PILRB:protein_coding|ENST00000448382.1:99949911:99965356:+:transcript:ENSG00000121716.14:PILRB:protein_coding|ENST00000448382.1:99964973:99965356:+:3UTR:ENSG00000121716.14:PILRB:protein_coding|ENST00000310771.4:99933737:99965454:+:transcript:ENSG00000272752.1:STAG3L5P-PVRIG2P-PILRB:processed_transcript|ENST00000310771.4:99964972:99965454:+:exon:ENSG00000272752.1:STAG3L5P-PVRIG2P-PILRB:processed_transcript|ENSG00000121716.14:99933737:99965356:+:gene:ENSG00000121716.14:PILRB:protein_coding|ENST00000452089.1:99949799:99965356:+:transcript:ENSG00000121716.14:PILRB:protein_coding|ENST00000452089.1:99964972:99965356:+:exon:ENSG00000121716.14:PILRB:protein_coding|ENST00000452089.1:99964998:99965356:+:3UTR:ENSG00000121716.14:PILRB:protein_coding|ENST00000610247.1:99933737:99965355:+:transcript:ENSG00000121716.14:PILRB:protein_coding|ENST00000610247.1:99964972:99965355:+:exon:ENSG00000121716.14:PILRB:protein_coding|ENST00000610247.1:99964998:99965355:+:3UTR:ENSG00000121716.14:PILRB:protein_coding|ENSG00000085514.11:99965153:99997719:+:gene:ENSG00000085514.11:PILRA:protein_coding|ENST00000444874.1:99933866:99965286:+:transcript:ENSG00000272752.1:STAG3L5P-PVRIG2P-PILRB:processed_transcript|ENST00000444874.1:99964972:99965286:+:exon:ENSG00000272752.1:STAG3L5P-PVRIG2P-PILRB:processed_transcript|ENSG00000272752.1:99933727:99965454:+:gene:ENSG00000272752.1:STAG3L5P-PVRIG2P-PILRB:processed_transcript|ENST00000444073.1:99964972:99965355:+:exon:ENSG00000121716.14:PILRB:protein_coding|ENST00000444073.1:99964998:99965355:+:3UTR:ENSG00000121716.14:PILRB:protein_coding|ENST00000444073.1:99951576:99965355:+:transcript:ENSG00000121716.14:PILRB:protein_coding,5UTR,0.000000,-0.321143,5utr_and_3utr|partial|ENSG00000085514.11:ENSG00000121716.14,ENSG00000121716.14|ENSG00000085514.11,5UTR_AND_3UTR
1017,chr7,138252225,138252226,0.000000,-0.321143,+,ENST00000343526.4:138252226:138252399:+:CDS:ENSG00000122779.12:TRIM24:protein_coding,"ENST00000493595.1:138252226:138252399:+:exon:ENSG00000122779.12:TRIM24:processed_transcript,protein_coding|ENST00000493595.1:138239515:138258387:+:transcript:ENSG00000122779.12:TRIM24:processed_transcript|ENST00000343526.4:138145079:138274738:+:transcript:ENSG00000122779.12:TRIM24:protein_coding|ENST00000343526.4:138252226:138252399:+:exon:ENSG00000122779.12:TRIM24:processed_transcript,protein_coding|ENST00000343526.4:138252226:138252399:+:CDS:ENSG00000122779.12:TRIM24:protein_coding|ENST00000497516.1:138252226:138252399:+:exon:ENSG00000122779.12:TRIM24:processed_transcript,protein_coding|ENST000004

In [99]:
wd = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
annotated_bedhead = ['chrom','start','end','p','f','strand','annotation','gene']
eric = pd.read_table(
    os.path.join(
        wd, 'ms1_ctrl_r1_01.basedon_ms1_ctrl_r1_01.peaks.l2inputnormnew.bed.compressed.bed.annotated'
    ),
    names=annotated_bedhead
)
eric[['chrom','start','end','p','f','strand']].to_csv('/home/bay001/projects/codebase/bfx/pyscripts/data/ms1_ctrl_r1_01.basedon_ms1_ctrl_r1_01.peaks.l2inputnormnew.bed.compressed.bed', sep='\t', index=False, header=False)

In [10]:
genes = gene_dict(features['chr7:104742399-104742504'])

NameError: name 'gene_dict' is not defined

In [ ]:
def transcript_dict(features, priority=PRIORITY):
    """
    Groups each transcript feature into genes.
    """
    transcripts = defaultdict(list)
    for feature in features:
        for transcript_id in feature.attributes['transcript_id']:
            transcripts[transcript_id].append(feature)
    for transcript, features in transcripts.iteritems():
        features.sort(key=lambda x: priority.index(x.featuretype.upper()))
    return transcripts

In [ ]:
from collections import defaultdict

# PRIORITY = ['UTR', 'CDS', 'utr3', 'EXON', 'GENE', 'TRANSCRIPT']
PRIORITY = ['CDS', 'UTR', 'GENE', 'EXON', 'TRANSCRIPT', 'START_CODON', 'STOP_CODON']

def prioritize(features, priority=PRIORITY):
    """
    # Groups each transcript feature into genes.
    """
    hi = [] # contains highest priority genic feature for each gene
    genes = defaultdict(list)
    for feature in features:
        for gene_id in feature.attributes['gene_id']:
            genes[gene_id].append(feature)
    for gene, features in genes.iteritems():
        features.sort(key=lambda x: priority.index(x.featuretype.upper()))
    for gene_id, overlapping_features in genes.iteritems():
        
        hi.append('{}|{}'.format(overlapping_features[0].featuretype.replace('gene','INTRON'), gene_id))
    return hi

In [ ]:
genes    def prioritize(
            self, features, v=True,
            transcript_priority=TRANSCRIPT_PRIORITY,
            gene_priority=GENE_PRIORITY
    ):
        """
        Groups each transcript feature into genes.
        """
        genic_features_reported = []
        genes = defaultdict(list)
        for feature in features:
            for gene_id in feature.attributes['gene_id']:
                genes[gene_id].append(feature) # append features to their respective genes

        for gene, features in genes.iteritems(): # for each transcript, iterate through feature list and prioritize
            for feature in features:
                if 'protein_coding' not in feature.attributes['transcript_type']:
                    if v == True:
                        print(feature.attributes['transcript_type'])
                    if feature.featuretype == 'exon' or feature.featuretype == 'UTR':
                        feature.featuretype = 'NONCODING_EXON'
                    elif feature.featuretype == 'gene':
                        feature.featuretype = 'NONCODING_INTRON'
                if 'UTR' == feature.featuretype:
                    feature.featuretype = self._classify_utr(feature)
            features.sort(
                key=lambda x: transcript_priority.index(x.featuretype.upper())
            )
        for gene_id, overlapping_features in genes.iteritems(): # iterate over each gene and prioritize according to genic region.
            # if v == True:
            #     print(gene_id)
            #     for feature in overlapping_features:
            #         print(feature.featuretype, feature.attributes['transcript_id'], feature.attributes['gene_id'], feature.start, feature.end)
            genic_features_reported.append(overlapping_features[0])

        genic_features_reported.sort(
            key=lambda x: gene_priority.index(x.featuretype.upper())
        )
        report = ('{}|{}'.format(genic_features_reported[0].featuretype, genic_features_reported[0].attributes['gene_id']))
        return report

In [ ]:
def prioritize(
            self, features, v=True, transcript_priority=PRIORITY):
        """
        Groups each transcript feature into genes.
        """
        hi = []  # contains highest priority genic feature for each gene
        genes = defaultdict(list)
        for feature in features:
            for gene_id in feature.attributes['gene_id']:
                genes[gene_id].append(feature) # append features to their respective genes

        for gene, features in genes.iteritems(): # for each transcript, iterate through feature list and prioritize
            for feature in features:
                if 'protein_coding' not in feature.attributes['transcript_type']:
                    if v == True:
                        print(feature.attributes['transcript_type'])
                    if feature.featuretype == 'exon' or feature.featuretype == 'UTR':
                        feature.featuretype = 'NONCODING_EXON'
                    elif feature.featuretype == 'gene':
                        feature.featuretype = 'NONCODING_INTRON'
            features.sort(key=lambda x: transcript_priority.index(x.featuretype.upper()))
        for gene_id, overlapping_features in genes.iteritems(): # iterate over each gene and prioritize according to genic region.
            if v == True:
                print(gene_id)
                for feature in overlapping_features:
                    print(feature.featuretype, feature.attributes['transcript_id'], feature.attributes['gene_id'], feature.start, feature.end)
            first_priority = overlapping_features[0]
            if first_priority.featuretype == 'UTR':
                annotation = self._classify_utr(first_priority)
            else:
                annotation = first_priority # .featuretype.replace('gene','INTRON')
            hi.append('{}|{}'.format(annotation, gene_id))
        return hi